In [250]:
import sys
sys.path.append('../')
from models.experimental import attempt_load
from models.yolo import Model,Detect
from models.common import Conv,C3,SPPF,Concat
import torch.nn as nn
import torch
import scipy.cluster.hierarchy as hac
from sklearn.preprocessing import normalize
from utils.general import non_max_suppression,check_img_size, scale_boxes, one_cycle
from utils.dataloaders import LoadImages
from utils.augmentations import letterbox,augment_hsv,random_perspective

from torch.utils.data import Dataset
from torchvision.transforms import ToTensor,Lambda
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler



import numpy as np
import yaml
import copy
import cv2
import pandas as pd
import json
from tqdm import tqdm
from pathlib import Path
from glob import glob
import random
import os
import matplotlib.pyplot as plt
%matplotlib inline
# !pip3 install --upgrade Pillow

In [251]:
# runs/train/prune-66-finetune-carpart-add-car-v2-anchor-tune-3-od-od/weights/best.pt
# runs/train/carpart-add-car-v2-js/weights/best.pt
model  = attempt_load('../runs/train/prune-66-finetune-carpart-add-car-v2-anchor-tune-3-od-od/weights/best.pt')

Fusing layers... 
YOLOv5s-anchortune-3 summary: 157 layers, 6040633 parameters, 0 gradients, 13.1 GFLOPs


In [252]:
model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 25, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(25, 55, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(55, 24, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(55, 27, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(51, 40, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(24, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [223]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [248]:
class Classify(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(229,128,1)
        self.act_conv = nn.SiLU() 
        self.pool = nn.AvgPool2d(2,2)
        
#         self.bn = nn.Bat
        
        self.ln1 = nn.Linear(3200,3200)
        self.act1 = nn.SiLU()
        self.drop1 = nn.Dropout(0.1)
        
        self.ln2 = nn.Linear(3200,2)
        
    def forward(self,x):
        x = self.act_conv(self.conv(x))
        x = self.pool(x)
        
        x = torch.flatten(x,1)
#         print(x.shape)
        x = self.drop1(self.act1(self.ln1(x)))
        
        return self.ln2(x)

class Classify_small_23(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(229,1,1)
        self.act_conv = nn.SiLU() 
        self.pool = nn.AvgPool2d(2,2)
        
#         self.bn = nn.Bat
        
        self.ln1 = nn.Linear(25,25)
        self.act1 = nn.SiLU()
        self.drop1 = nn.Dropout(0.1)
        
        self.ln2 = nn.Linear(25,2)
        
    def forward(self,x):
        x = self.act_conv(self.conv(x))
#         print(x.shape)
        x = self.pool(x)
#         print('pool : ',x.shape)
        
        x = torch.flatten(x,1)
        x = self.drop1(self.act1(self.ln1(x)))
        
        return self.ln2(x)

class Classify_small_17(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(85,1,1)
        self.act_conv = nn.SiLU() 
        self.pool = nn.AvgPool2d(2,2)
        
#         self.bn = nn.Bat
        
        self.ln1 = nn.Linear(400,400)
        self.act1 = nn.SiLU()
        self.drop1 = nn.Dropout(0.1)
        
        self.ln2 = nn.Linear(400,2)
        
    def forward(self,x):
        x = self.act_conv(self.conv(x))
#         print(x.shape)
        x = self.pool(x)
#         print('pool : ',x.shape)
        
        x = torch.flatten(x,1)
        x = self.drop1(self.act1(self.ln1(x)))
        
        return self.ln2(x)
    
class Warper_model(nn.Module):
    def __init__(self,yolo_model,classify_head):
        self.yolo_model = yolo_model
        self.classify_head = classify_head
        self.hook_data = {}
        
        def get_hook_data(name):
            def hook(model, input, output):
                self.hook_data[name] = output.detach()
            return hook
        self.yolo_model.model[17].cv3.act.register_forward_hook(get_hook_data('17c3act'))
        
    def forward(self,x):
        x = x.float()/255.0
        
        logit_yolo = self.yolo_model(x)
        input_head = self.hook_data['17c3act']
        logit = self.classify_head(input_head)
        
        return logit
        

class_head = Classify_small_23()

In [233]:
x= torch.randn(1,3,320,320)

In [234]:
model  = attempt_load('../runs/train/prune-66-finetune-carpart-add-car-v2-anchor-tune-3-od-od/weights/best.pt')
hook_data = {}
def get_hook_data(name):
    def hook(model, input, output):
        hook_data[name] = output.detach()
#         print(hook_data[name].sum())
    return hook

model.model[23].cv3.register_forward_hook(get_hook_data('17c3act'))

out_x = model(x)
in_classify = hook_data['17c3act']
# print(in_classify.shape)
act = nn.SiLU(inplace=True)
print(act(in_classify).shape)
# print(class_head(in_classify).shape)

# tensor(3996.33325)
# tensor(3865.61987)

Fusing layers... 
YOLOv5s-anchortune-3 summary: 157 layers, 6040633 parameters, 0 gradients, 13.1 GFLOPs


torch.Size([1, 229, 10, 10])


In [227]:
model.model[17]

C3(
  (cv1): Conv(
    (conv): Conv2d(230, 60, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (cv2): Conv(
    (conv): Conv2d(230, 62, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (cv3): Conv(
    (conv): Conv2d(122, 85, kernel_size=(1, 1), stride=(1, 1))
    (act): SiLU(inplace=True)
  )
  (m): Sequential(
    (0): Bottleneck(
      (cv1): Conv(
        (conv): Conv2d(60, 64, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 60, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
    )
  )
)

In [89]:
model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 25, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(25, 55, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(55, 24, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(55, 27, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(51, 40, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(24, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  

In [50]:
train = pd.read_csv('train.csv', index_col=0)

train

,file_name,label
0,../coco/scratch/non_zoomin/https:__mc-vip.s3-a...,0
1,../coco/scratch/non_zoomin/https:__mc-vip.s3-a...,0
2,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1
3,../coco/scratch/non_zoomin/https:__mc-vip.s3-a...,0
4,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1
...,...,...
7703,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1
7704,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1
7705,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1
7706,../coco/scratch/zoomin/https:__mc-vip.s3-ap-so...,1


In [245]:
# data_path = Path('../coco/scratch')
# zoomin_files = glob('../coco/scratch/zoomin/*')
# non_zoomin_files = glob('../coco/scratch/non_zoomin/*')

# labels = [1 for i in range(len(zoomin_files))]+[0 for i in range(len(non_zoomin_files))]
# files = zoomin_files+non_zoomin_files

# d = {'file_name':files,'label':labels}
# df = pd.DataFrame(d)
# df= df.sample(frac=1).reset_index(drop=True)
# # df.to_csv('zoomin_data.csv')
# # print(df)
# num = df.shape[0]
# train = df.iloc[:int(0.8*num)]
# val = df.iloc[int(0.8*num):].reset_index(drop=True)

train = pd.read_csv('train.csv', index_col=0)
val = pd.read_csv('valid.csv',index_col=0)

# print(val)
class ZoominDataset(Dataset):
    def __init__(self,label_csv,transform,imgsz=320,is_train=True):
        self.csv = label_csv
        self.img_size = imgsz
        self.transform = transform
        self.is_train = is_train 
        
    def __len__(self):
        return self.csv.shape[0]
    
    def __getitem__(self,idx):
        filename = self.csv.iloc[idx,0]
#         print(filename)
        im = cv2.imread(filename)

        h0, w0 = im.shape[:2]  # orig hw
        r = self.img_size / max(h0, w0)  # ratio
        if r != 1:  # if sizes are not equal
            interp = cv2.INTER_LINEAR 
            im = cv2.resize(im, (int(w0 * r), int(h0 * r)), interpolation=interp)
#         (h0, w0), im.shape[:2]
        im,_,_ = letterbox(im,self.img_size,auto=False)
        
        if self.is_train : 
            augment_hsv(im,0.015,0.7,0.4)
            if random.random() < 0.5 :
                im = np.fliplr(im)

            if random.random() < 0.1 : 
                im,_ = random_perspective(im)
        
#         if self.transform : 
#             im = self.transform(im.copy())
        im = im[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
        im = np.ascontiguousarray(im)
            
    
        label = self.csv.iloc[idx,1]
        
    
        
        return torch.from_numpy(im),label

In [246]:
train_dataloader = DataLoader(ZoominDataset(train,ToTensor(),is_train=True),batch_size=16,shuffle=True)
val_dataloader = DataLoader(ZoominDataset(val,ToTensor(),is_train=False),batch_size=16,shuffle=False)

In [237]:
def train_loop(yolo_model,classify_head,train_loader, val_loader,loss_function,optimizer,scheduler,epochs):
    yolo_model.to(device)
    classify_head.to(device)
    classify_head.train()
    
    best_acc = 0
    best_model = copy.deepcopy(classify_head)
    for e in range(epochs):
        classify_head.train()
        for inp,label in tqdm(train_loader,desc='epoch '+str(e)):
            optimizer.zero_grad()
            
            inp = inp.to(device).float() / 255
            label = label.to(device)
            
            logit_yolo = yolo_model(inp)
            input_head = hook_data['17c3act']
            logit = classify_head(input_head)
            #rint(label,logit)
            loss = loss_function(logit,label)
#             print(logit.sum(),loss)
            
            loss.backward()
            optimizer.step()
            scheduler.step()
        acc = validate(yolo_model,classify_head,val_loader)
        print('acc : ',acc)
        if acc > best_acc:
            best_acc = acc
            best_model = copy.deepcopy(classify_head)
            torch.save(classify_head.state_dict(),'zoomin_model.pth')
    
    return best_model

softmax = nn.Softmax(dim=1)
def validate(yolo_model,classify_head,val_loader):
    classify_head.eval()
    classify_head.to(device)
    yolo_model.to(device)
    leng = 0
    
    correct = 0
    for inp, label in tqdm(val_loader,desc='val .... '):
        inp = inp.to(device).float() / 255
#         label = label

        logit_yolo = yolo_model(inp)
        input_head = hook_data['17c3act']
        logit = classify_head(input_head)
#         print(logit)

        pred = np.argmax(softmax(logit).cpu().detach().numpy(),axis=1)
        correct += sum(label.numpy()==pred)
        leng += len(pred)
#         print(pred,correct)
#     print('correct : ',correct)
#     print('leng : ',leng)
    return correct/leng

In [239]:
class_head

Classify_small_23(
  (conv): Conv2d(229, 1, kernel_size=(1, 1), stride=(1, 1))
  (act_conv): SiLU()
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (ln1): Linear(in_features=25, out_features=25, bias=True)
  (act1): SiLU()
  (drop1): Dropout(p=0.1, inplace=False)
  (ln2): Linear(in_features=25, out_features=2, bias=True)
)

In [249]:
epochs = 200
class_head.to(device)
class_head.train()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(class_head.parameters(), lr=0.001, momentum=0.9)

lf = one_cycle(1, 0.1, epochs)
scheduler = lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=lf)
# for k, v in class_head.named_parameters():
#     print(v.requires_grad)
# load pretrain 
# checkpoint = torch.load('zoomin_model_exp1.pth')
# class_head.load_state_dict(checkpoint)

best_model = train_loop(model,class_head,train_dataloader,val_dataloader,criterion,optimizer_ft,scheduler,epochs)

val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:33<00:00,  3.65it/s]


acc :  0.9252724442138038


val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:33<00:00,  3.60it/s]


acc :  0.9372080954852102


val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:33<00:00,  3.58it/s]


...

acc :  0.9782044628956927


val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:31<00:00,  3.82it/s]


acc :  0.9792423456149455


val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 121/121 [00:31<00:00,  3.81it/s]

acc :  0.9792423456149455


In [107]:
checkpoint = torch.load('zoomin_model_exp1.pth')
class_head.load_state_dict(checkpoint)

model.to(device)
validate(model,class_head,val_dataloader)

val .... :   1%|█                                                                                                                              | 2/241 [00:00<00:27,  8.58it/s]

torch.Size([8, 1, 10, 10])
tensor([[ 1.61971, -1.84860],
        [ 2.45281, -2.68875],
        [-1.47191,  1.62443],
        [ 0.28240, -0.32831],
        [ 1.07322, -1.25128],
        [ 0.40979, -0.46806],
        [ 0.85750, -1.05445],
        [-2.03759,  2.12274]], device='cuda:0', grad_fn=<AddmmBackward>)
[0 0 1 0 0 0 0 1] 7
torch.Size([8, 1, 10, 10])
tensor([[-0.34342,  0.14321],
        [-0.67565,  0.61796],
        [-0.33949,  0.26472],
        [-4.83991,  5.26159],
        [-2.80397,  2.90802],
        [ 1.28354, -1.47561],
        [ 0.03832, -0.08999],
        [ 1.69920, -1.86193]], device='cuda:0', grad_fn=<AddmmBackward>)
[1 1 1 1 1 0 0 0] 12
torch.Size([8, 1, 10, 10])
tensor([[ 1.34415, -1.51400],
        [ 1.78466, -2.03559],
        [-0.71248,  0.73051],
        [ 1.52727, -1.76691],
        [ 2.15048, -2.31535],
        [-1.46733,  1.29128],
        [-2.04313,  2.16505],
        [ 0.80577, -0.92140]], device='cuda:0', grad_fn=<AddmmBackward>)
[0 0 1 0 0 1 1 0] 19


val .... :   2%|██▋                                                                                                                            | 5/241 [00:00<00:34,  6.83it/s]

torch.Size([8, 1, 10, 10])
tensor([[-1.28463,  1.28989],
        [ 1.33211, -1.56546],
        [ 0.45618, -0.47310],
        [ 1.81030, -2.07255],
        [-3.94656,  4.35268],
        [-4.53003,  4.17150],
        [-0.14088, -0.04730],
        [ 0.11760, -0.33938]], device='cuda:0', grad_fn=<AddmmBackward>)
[1 0 0 0 1 1 1 0] 26
torch.Size([8, 1, 10, 10])
tensor([[ 1.57793, -1.79610],
        [-5.11788,  5.25977],
        [-0.93992,  0.76414],
        [-3.05511,  3.43978],
        [-1.91762,  1.97874],
        [ 0.53733, -0.69960],
        [-2.41574,  2.61770],
        [ 0.65165, -0.75949]], device='cuda:0', grad_fn=<AddmmBackward>)
[0 1 1 1 1 0 1 0] 33


val .... :   2%|███▏                                                                                                                           | 6/241 [00:00<00:37,  6.23it/s]

...

val .... :  99%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 239/241 [00:34<00:00,  6.50it/s]

torch.Size([8, 1, 10, 10])
tensor([[-1.63142,  1.59637],
        [-0.87121,  0.56772],
        [ 1.33268, -1.46776],
        [ 1.25342, -1.50460],
        [-0.87841,  0.68872],
        [ 2.09015, -2.25061],
        [-1.02472,  0.95106],
        [ 0.11404, -0.32741]], device='cuda:0', grad_fn=<AddmmBackward>)
[1 1 0 0 1 0 1 0] 1643
torch.Size([8, 1, 10, 10])
tensor([[ 1.08008, -1.22079],
        [-1.18989,  1.21816],
        [ 0.16606, -0.36895],
        [-3.46051,  3.78452],
        [ 2.35452, -2.46205],
        [-0.43370,  0.41613],
        [-1.10946,  1.11151],
        [ 1.05450, -1.25601]], device='cuda:0', grad_fn=<AddmmBackward>)
[0 1 0 1 0 1 1 0] 1650


val .... : 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 241/241 [00:34<00:00,  6.93it/s]

torch.Size([8, 1, 10, 10])
tensor([[ 0.85288, -1.14320],
        [-2.25352,  2.16076],
        [ 1.86126, -2.05356],
        [ 0.60799, -0.87571],
        [ 0.45348, -0.64977],
        [-0.38903,  0.22006],
        [-1.03520,  0.88417],
        [-1.84767,  1.77745]], device='cuda:0', grad_fn=<AddmmBackward>)
[0 1 0 0 0 1 1 1] 1655
torch.Size([7, 1, 10, 10])
tensor([[-1.36915,  1.51671],
        [ 0.27259, -0.46564],
        [-0.77187,  0.83403],
        [ 0.13272, -0.33744],
        [ 0.81577, -0.70313],
        [-3.14932,  3.39883],
        [-2.46933,  2.65976]], device='cuda:0', grad_fn=<AddmmBackward>)
[1 0 1 0 0 1 1] 1661


0.8619615983393877

In [15]:
# !cp zoomin_model.pth zoomin_model_exp1.pth
!ls

 Untitled.ipynb       'review hibrid.ipynb'    zoomin_data.csv
 debug-model.ipynb     runs		       zoomin_model.pth
 not-zoomin.txt        tutorial.ipynb	       zoomin_model_exp1.pth
 prune.ipynb	       update_carpart.ipynb
 quantiazation.ipynb   zoomin-classify.ipynb


In [21]:
! cp ../coco/scratch/gen_images/https:__mc-vip.s3-ap-southeast-1.amazonaws.com_krug_krug-01-10k-img_SVP390060_R90.jpg ../coco/scratch/non_zoomin/

In [22]:
cv2.imread('../coco/scratch/non_zoomin/https:__mc-vip.s3-ap-southeast-1.amazonaws.com_krug_krug-01-10k-img_SVP390060_R90.jpg')

array([[[ 24,  28,  29],
        [ 24,  28,  29],
        [ 25,  29,  30],
        ...,
        [ 16,   0, 143],
        [ 19,   0, 149],
        [ 17,   0, 151]],

       [[ 24,  28,  29],
        [ 25,  29,  30],
        [ 26,  30,  31],
        ...,
        [ 28,   6, 148],
        [ 28,   7, 153],
        [ 25,   6, 153]],

       [[ 23,  27,  28],
        [ 24,  28,  29],
        [ 26,  30,  31],
        ...,
        [ 38,  13, 147],
        [ 38,  13, 151],
        [ 33,  10, 149]],

       ...,

       [[157, 169, 179],
        [156, 173, 182],
        [168, 194, 201],
        ...,
        [191, 214, 222],
        [175, 196, 204],
        [168, 189, 197]],

       [[185, 201, 208],
        [185, 204, 211],
        [193, 219, 225],
        ...,
        [185, 211, 218],
        [183, 209, 216],
        [188, 214, 221]],

       [[140, 156, 162],
        [159, 181, 186],
        [166, 195, 199],
        ...,
        [162, 190, 197],
        [165, 194, 201],
        [157, 186, 193]]

In [10]:
# data = json.load(open('../coco/scratch/annotations/carpart-gen.json'))
# print(len(data['images']))

In [109]:
# annos_by_img_ids = {}
# for i in tqdm(data['images']):
#     annos_by_img_ids[i['id']] = [a for a in data['annotations'] if a['image_id'] == i['id']]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51965/51965 [2:24:34<00:00,  5.99it/s]


In [ ]:
# for i in tqdm(data['images']):
#     if len(annos_by_img_ids[i['id']]) > 8 :
#         os.system('cp ../coco/scratch/gen_images/'+i['file_name']+ ' '+str(non_zoomin_folder))
#         with open('not-zoomin.txt','a') as f :
#             f.write(i['file_name']+'\n')
    

 11%|███████████████▎                                                                                                                               | 5579/51965 [01:05<26:31, 29.15it/s]

In [116]:
# non_zoomin_folder = Path('../coco/scratch/non_zoomin')
# zoomin_folder = Path('../coco/scratch/zoomin')
# non_zoomin_folder.mkdir(parents=True, exist_ok =True)
# zoomin_folder.mkdir(parents=True,exist_ok=True)

In [125]:
## move zoomin image
# files = glob('../coco/scratch/crop_gen_images/*')
# print(len(files))
# for file in files[:5000]:
#     os.system('cp '+str(file)+ ' '+str(zoomin_folder))

31980


In [23]:
# train.to_csv('train.csv')
# val.to_csv('valid.csv')

In [109]:
model.state_dict

<bound method Module.state_dict of DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 25, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(25, 55, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(55, 24, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(55, 27, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv3): Conv(
        (conv): Conv2d(51, 40, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): Sequential(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(24, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(32, 24, kernel_size=(3, 3)